In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import optuna

In [2]:
Ridership_df= pd.read_csv('data/Ridership.csv')
df= Ridership_df.copy()
df.sample(5)

,Year,Month,Day,Week Number,Corridor,Workday,Station,Period,Ridership,N_trains,Covid19
62858,2022,November,30,48,Corridor_1,y,Station_3,PM Peak,2516,5,0
8942,2019,July,3,27,Corridor_1,y,Station_2,PM Peak,61,1,0
40936,2021,August,9,32,Corridor_3,y,Station_3,PM Peak,1252,12,1
1242,2019,January,23,4,Corridor_3,y,Station_4,Midday,3824,12,0
2610,2019,February,17,7,Corridor_1,n,Station_2,Weekend/Holiday,394,10,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64369 entries, 0 to 64368
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         64369 non-null  int64 
 1   Month        64369 non-null  object
 2   Day          64369 non-null  int64 
 3   Week Number  64369 non-null  int64 
 4   Corridor     64369 non-null  object
 5   Workday      64369 non-null  object
 6   Station      64369 non-null  object
 7   Period       64369 non-null  object
 8   Ridership    64369 non-null  int64 
 9   N_trains     64369 non-null  int64 
 10  Covid19      64369 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 5.4+ MB


In [4]:
df.duplicated().sum()

np.int64(0)

In [5]:
df.isnull().sum()

Year           0
Month          0
Day            0
Week Number    0
Corridor       0
Workday        0
Station        0
Period         0
Ridership      0
N_trains       0
Covid19        0
dtype: int64

# Feature Engineering

This section defines a function feature_engineering that encapsulates several crucial data transformation steps. These steps prepare the raw Ridership data into a format suitable for machine learning models, enhancing their ability to learn patterns from the metro service data.

In [6]:
def feature_engineering(df):
    # Month mapping
    month_mapping = {
        'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
        'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
    }
    df['Month_Num'] = df['Month'].map(month_mapping)

    # Cyclical Day conversion
    def convert_day_to_circle(day):
        angle = 2 * np.pi * (day - 1) / 31
        x = np.cos(angle)
        y = np.sin(angle)
        return x, y

    df['day_x'], df['day_y'] = zip(*df['Day'].map(convert_day_to_circle))

    # Cyclical Week conversion
    def convert_week_to_circle(week):
        angle = 2 * np.pi * (week - 1) / 53
        x = np.cos(angle)
        y = np.sin(angle)
        return x, y

    df['week_x'], df['week_y'] = zip(*df['Week Number'].map(convert_week_to_circle))

    # Categorical features for OneHotEncoder
    categorical_features = ['Month', 'Corridor', 'Workday', 'Station', 'Period']

    # Column transformer for preprocessing
    preprocessor = ColumnTransformer(
        transformers=[
            ('', OneHotEncoder(), categorical_features),
        ], remainder='passthrough'
    )

    # Fit and transform the DataFrame
    df_transformed = preprocessor.fit_transform(df)

    # Convert the encoded features to a dense matrix
    df_transformed = df_transformed.toarray()

    # Convert to DataFrame and get feature names
    feature_names_out = list(preprocessor.get_feature_names_out())
    
    # Convert the dense matrix to a DataFrame
    df_transformed = pd.DataFrame(df_transformed, columns=[item.split('__')[1] for item in feature_names_out])

    return df_transformed

In [7]:
df= feature_engineering(df)

In [8]:
df['COVID_Workday'] = df['Covid19'] * df['Workday_y']

columns_to_convert = [
    'Month_April', 'Month_August', 'Month_December', 'Month_February', 
    'Month_January', 'Month_July', 'Month_June', 'Month_March', 
    'Month_May', 'Month_November', 'Month_October', 'Month_September',
    'Corridor_Corridor_1', 'Corridor_Corridor_2', 'Corridor_Corridor_3', 
    'Corridor_Corridor_4', 'Corridor_Corridor_5', 'Corridor_Corridor_6', 'Corridor_Corridor_7',
    'Station_Station_1', 'Station_Station_10', 'Station_Station_11', 'Station_Station_12', 
    'Station_Station_13', 'Station_Station_14', 'Station_Station_15', 'Station_Station_16', 
    'Station_Station_17', 'Station_Station_18', 'Station_Station_19', 'Station_Station_2', 
    'Station_Station_20', 'Station_Station_21', 'Station_Station_22', 'Station_Station_23', 
    'Station_Station_24', 'Station_Station_25', 'Station_Station_26', 'Station_Station_27', 
    'Station_Station_28', 'Station_Station_29', 'Station_Station_3', 'Station_Station_30', 
    'Station_Station_31', 'Station_Station_32', 'Station_Station_33', 'Station_Station_34', 
    'Station_Station_35', 'Station_Station_36', 'Station_Station_37', 'Station_Station_38', 
    'Station_Station_39', 'Station_Station_4', 'Station_Station_40', 'Station_Station_41', 
    'Station_Station_42', 'Station_Station_43', 'Station_Station_44', 'Station_Station_45', 
    'Station_Station_5', 'Station_Station_6', 'Station_Station_7', 'Station_Station_8', 'Station_Station_9',
    'Period_AM Peak', 'Period_Evening', 'Period_Midday', 'Period_PM Peak', 'Period_Weekend/Holiday',
    'Day',
    'Week Number',
    'Workday_n', 'Workday_y',
    'Covid19','COVID_Workday',
    'Month_Num']

columns_to_convert_02=['Year', 'Ridership', 'N_trains']


df[columns_to_convert] = df[columns_to_convert].astype('uint8')
df[columns_to_convert_02] = df[columns_to_convert_02].astype('uint16')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64369 entries, 0 to 64368
Data columns (total 83 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Month_April             64369 non-null  uint8  
 1   Month_August            64369 non-null  uint8  
 2   Month_December          64369 non-null  uint8  
 3   Month_February          64369 non-null  uint8  
 4   Month_January           64369 non-null  uint8  
 5   Month_July              64369 non-null  uint8  
 6   Month_June              64369 non-null  uint8  
 7   Month_March             64369 non-null  uint8  
 8   Month_May               64369 non-null  uint8  
 9   Month_November          64369 non-null  uint8  
 10  Month_October           64369 non-null  uint8  
 11  Month_September         64369 non-null  uint8  
 12  Corridor_Corridor_1     64369 non-null  uint8  
 13  Corridor_Corridor_2     64369 non-null  uint8  
 14  Corridor_Corridor_3     64369 non-null

# Problem One: 
Assume we want to find a general model for the number of passengers without looking at the number of passengers on previous days and only using the recorded information in the table for the same time period (except for the year and number of trains) to predict the number of required trains based on the number of passengers.

## Splitting data for problem One :

This section defines the preprocess_and_split_data_p01 function, which is responsible for preparing the data and splitting it into training and testing sets specifically for Problem One: finding a general model for passenger prediction.

In [10]:
def preprocess_and_split_data_p01(df, target_column='Ridership', test_size=0.2, random_state=42):
    # Drop unnecessary columns
    X = df.drop([target_column, 'Year', 'N_trains'], axis=1)
    y = df[target_column]

    # Create stratification key directly from df
    df_for_stratify_p01 = Ridership_df.copy()
    stratify_key_p01 = df_for_stratify_p01['Covid19'].astype(str) + '_' + \
                       df_for_stratify_p01['Workday'] + '_' + \
                       df_for_stratify_p01['Period']+ '_' + \
                       df_for_stratify_p01['Corridor']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        random_state=random_state,
        stratify=stratify_key_p01)
    
    print(f"\nTrain set features (X_train) shape: {X_train.shape}")
    print(f"Test set features (X_test) shape: {X_test.shape}")
    print(f"Train set target (y_train) shape: {y_train.shape}")
    print(f"Test set target (y_test) shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test

In [11]:
X_train_p01, X_test_p01, y_train_p01, y_test_p01 = preprocess_and_split_data_p01(df)


Train set features (X_train) shape: (51495, 80)
Test set features (X_test) shape: (12874, 80)
Train set target (y_train) shape: (51495,)
Test set target (y_test) shape: (12874,)


### LightGBM model:

This code defines the objective_p01 function, which serves as the core of the hyperparameter optimization process using the Optuna library for Problem One. Optuna calls this function repeatedly, each time suggesting a new set of hyperparameters for the LightGBM model, trains the model, and then evaluates its performance. Optuna then uses this feedback to intelligently propose better parameters in subsequent trials.

In [12]:
def objective_p01(trial):
    
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'verbose': -1,
        'feature_pre_filter': False 
    }


    train_data = lgb.Dataset(X_train_p01, label=y_train_p01)
    test_data = lgb.Dataset(X_test_p01, label=y_test_p01, reference=train_data)

    model = lgb.train(params,
                      train_data,
                      num_boost_round=1000,
                      valid_sets=[test_data],
                      callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]) 

    y_pred = model.predict(X_test_p01, num_iteration=model.best_iteration)
    return np.sqrt(mean_squared_error(y_test_p01, y_pred))

This code block initiates and executes the hyperparameter optimization process for Problem One using the Optuna library. It orchestrates the search for the best set of LightGBM hyperparameters that minimize the Root Mean Squared Error (RMSE) on the validation set.

In [13]:
study_p01 = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42)) 
study_p01.optimize(objective_p01, n_trials=50, show_progress_bar=True)

best_params_p01 = study_p01.best_params
best_rmse_optuna_p01 = study_p01.best_value
print("\nBest parameters found by Optuna:", best_params_p01)
print(f"Best RMSE on validation set during tuning (Optuna): {best_rmse_optuna_p01:.2f}")

[I 2025-07-22 03:27:06,335] A new study created in memory with name: no-name-2ba9b8cd-2c9a-4929-8e22-87d33e0a49f9


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-07-22 03:27:09,056] Trial 0 finished with value: 858.3986297975716 and parameters: {'num_leaves': 50, 'learning_rate': 0.28570714885887566, 'min_child_samples': 75, 'max_depth': 8, 'feature_fraction': 0.5780093202212182, 'bagging_fraction': 0.5779972601681014, 'bagging_freq': 0, 'lambda_l1': 8.661761457749352, 'lambda_l2': 6.011150117432088}. Best is trial 0 with value: 858.3986297975716.
[I 2025-07-22 03:27:13,478] Trial 1 finished with value: 965.171480859766 and parameters: {'num_leaves': 77, 'learning_rate': 0.01596950334578271, 'min_child_samples': 98, 'max_depth': 11, 'feature_fraction': 0.6061695553391381, 'bagging_fraction': 0.5909124836035503, 'bagging_freq': 2, 'lambda_l1': 3.0424224295953772, 'lambda_l2': 5.247564316322379}. Best is trial 0 with value: 858.3986297975716.
[I 2025-07-22 03:27:15,510] Trial 2 finished with value: 1000.1536696220307 and parameters: {'num_leaves': 54, 'learning_rate': 0.09445645065743215, 'min_child_samples': 63, 'max_depth': 4, 'feature_

The output provides the Best parameters found by Optuna and the Best RMSE on validation set during tuning.

This code block focuses on training the final LightGBM model for Problem One (general passenger prediction) using the optimal hyperparameters found by Optuna, and then evaluating its performance on the held-out test set.

In [14]:
final_model_p01 = lgb.train(best_params_p01,
                            lgb.Dataset(X_train_p01, label=y_train_p01),
                            num_boost_round=1000) 

y_pred_final_p01 = final_model_p01.predict(X_test_p01)

rmse_final_p01 = np.sqrt(mean_squared_error(y_test_p01, y_pred_final_p01))
r2_final_p01 = r2_score(y_test_p01, y_pred_final_p01)

print(f"\nFinal Model Performance on X_test_p01:")
print(f"RMSE: {rmse_final_p01:.2f}")
print(f"R²: {r2_final_p01:.4f}")


Final Model Performance on X_test_p01:
RMSE: 818.90
R²: 0.8157


#### ``RMSE: 818.90``
This means, on average, this model predictions for passenger numbers are off by approximately 819 passengers.

#### ``R²: 0.8157 ``
This indicates that approximately ``81.57%`` of the variance in passenger numbers in the test set can be explained by this model.

This code defines the calculate_needed_trains function, a crucial component for translating predicted passenger numbers into actionable operational decisions: the number of trains required. 
The primary goal of this function is to determine the optimal number of trains needed to accommodate a given number of predicted passengers, taking into account the train's capacity and optional buffering for comfort or safety.

In [17]:
def calculate_needed_trains(predicted_passengers_series: pd.Series,
                            train_capacity: int = 600,
                            safety_buffer_percentage: float = 0.05,
                            minimum_trains_required: int = 1) -> pd.DataFrame:

    predicted_passengers_int = predicted_passengers_series.astype(int)

    result_df = pd.DataFrame({
        'Predicted_Passengers': predicted_passengers_int})

    # 1. Calculate the base number of trains needed (minimum required)
    # np.ceil ensures that any fraction of a train needed results in a full additional train.
    result_df['Base_Predicted_Trains'] = np.ceil(
        result_df['Predicted_Passengers'] / train_capacity
    ).astype(int)

    # 2. Calculate trains with a safety/comfort buffer
    # This adds a percentage buffer to the predicted passengers before dividing by capacity.
    result_df['Buffered_Predicted_Trains'] = np.ceil(
        result_df['Predicted_Passengers'] * (1 + safety_buffer_percentage) / train_capacity
    ).astype(int)

    # 3. Ensure a minimum number of trains
    # This ensures that even for very low passenger counts, a base service level is maintained.
    result_df['Final_Predicted_Trains'] = result_df['Buffered_Predicted_Trains'].apply(
        lambda x: max(x, minimum_trains_required)
    )

    return result_df

In [18]:
y_pred_train_p01= calculate_needed_trains(y_pred_final_p01)
y_pred_train_p01.sample(10)

,Predicted_Passengers,Base_Predicted_Trains,Buffered_Predicted_Trains,Final_Predicted_Trains
1253,8576,15,16,16
7091,129,1,1,1
7473,484,1,1,1
6851,193,1,1,1
12709,390,1,1,1
4285,519,1,1,1
12204,2134,4,4,4
8419,1159,2,3,3
3503,447,1,1,1
11769,4072,7,8,8


# Problem Two:
 Assume we need to forecast the number of passengers for different time periods in order to allocate the appropriate number of trains one week in advance. (In this case, we are allowed to use information from previous time periods, but using the year and number of trains columns is still not permitted.)

## Splitting data for problem two :

### LightGBM model:

This code defines the preprocess_and_split_data_p02 function, which handles the data preparation and chronological splitting specifically for Problem Two: forecasting passenger numbers one week in advance. This problem requires a time-series approach, making the order of data crucial.
The function takes your processed DataFrame, creates time-series specific features (lags), and then splits the data into training and testing sets based on time, ensuring that the model only learns from past information to predict the future.

In [19]:
def preprocess_and_split_data_p02(df, target_column='Ridership', train_size=0.8):

    df['Date'] = pd.to_datetime(
        df['Year'].astype(str) + '-' +
        df['Month_Num'].astype(str) + '-' +
        df['Day'].astype(str),
        errors='coerce')

    df.dropna(subset=['Date'], inplace=True)

    # Grouping columns for creating lagged features
    grouping_cols = [
        col for col in df.columns if col.startswith('Corridor_') or
        col.startswith('Station_') or
        col.startswith('Period_')
    ]

    # Create lagged features for 'Passengers'
    # Lag 7 days (for same day of week, same period, etc., one week prior)
    df['Ridership_Lag_7'] = df.groupby(grouping_cols)['Ridership'].shift(7)

    # Lag 14 days (for same day of week, same period, two weeks prior)
    df['Ridership_Lag_14'] = df.groupby(grouping_cols)['Ridership'].shift(14)

    df.dropna(subset=['Ridership_Lag_7', 'Ridership_Lag_14'], inplace=True)    
    df.reset_index(drop=True, inplace=True)
    df.sort_values(by=['Date'], inplace=True)

    # Define features (X) and target (y)
    X = df.drop(columns=[target_column, 'Year', 'N_trains', 'Date'])
    y = df[target_column]

    # Calculate split point
    split_point = int(len(df) * train_size)

    # Split the data into training and testing sets
    X_train = X.iloc[:split_point]
    y_train = y.iloc[:split_point]
    
    X_test = X.iloc[split_point:]
    y_test = y.iloc[split_point:]

    print(f"\nTrain set features (X_train) shape: {X_train.shape}")
    print(f"Test set features (X_test) shape: {X_test.shape}")
    print(f"Train set target (y_train) shape: {y_train.shape}")
    print(f"Test set target (y_test) shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test


In [20]:
X_train_p02, X_test_p02, y_train_p02, y_test_p02 = preprocess_and_split_data_p02(df)


Train set features (X_train) shape: (49951, 82)
Test set features (X_test) shape: (12488, 82)
Train set target (y_train) shape: (49951,)
Test set target (y_test) shape: (12488,)


This code defines the objective_p02 function, which serves as the core of the hyperparameter optimization process using the Optuna library for Problem Two: forecasting passenger numbers one week in advance. Optuna repeatedly calls this function, suggesting new sets of hyperparameters for the LightGBM model, trains the model, and evaluates its performance on a validation set.

In [22]:
def objective_p02(trial):

    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'verbose': -1,
        'feature_pre_filter': False 
    }


    train_data = lgb.Dataset(X_train_p02, label=y_train_p02)
    test_data = lgb.Dataset(X_test_p02, label=y_test_p02, reference=train_data)

    model = lgb.train(params,
                      train_data,
                      num_boost_round=1000, 
                      valid_sets=[test_data],
                      callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]) 

    y_pred = model.predict(X_test_p02, num_iteration=model.best_iteration)

    return np.sqrt(mean_squared_error(y_test_p02, y_pred))

This code block initiates and executes the hyperparameter optimization process for Problem Two (forecasting one week in advance) using the Optuna library. It orchestrates the search for the best set of LightGBM hyperparameters that minimize the Root Mean Squared Error (RMSE) on the validation set

In [23]:
study_p02 = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42)) 
study_p02.optimize(objective_p02, n_trials=70, show_progress_bar=True) 


best_params_p02 = study_p02.best_params
best_rmse_optuna_p02 = study_p02.best_value
print("\nBest parameters found by Optuna:", best_params_p02)
print(f"Best RMSE on validation set during tuning (Optuna): {best_rmse_optuna_p02:.2f}")

[I 2025-07-22 04:14:05,514] A new study created in memory with name: no-name-67be581c-db7d-470f-b31b-ab010da4f090


  0%|          | 0/70 [00:00<?, ?it/s]

[I 2025-07-22 04:14:06,219] Trial 0 finished with value: 749.983216692356 and parameters: {'num_leaves': 50, 'learning_rate': 0.28570714885887566, 'min_child_samples': 75, 'max_depth': 8, 'feature_fraction': 0.5780093202212182, 'bagging_fraction': 0.5779972601681014, 'bagging_freq': 0, 'lambda_l1': 8.661761457749352, 'lambda_l2': 6.011150117432088}. Best is trial 0 with value: 749.983216692356.
[I 2025-07-22 04:14:10,177] Trial 1 finished with value: 689.8169979283266 and parameters: {'num_leaves': 77, 'learning_rate': 0.01596950334578271, 'min_child_samples': 98, 'max_depth': 11, 'feature_fraction': 0.6061695553391381, 'bagging_fraction': 0.5909124836035503, 'bagging_freq': 2, 'lambda_l1': 3.0424224295953772, 'lambda_l2': 5.247564316322379}. Best is trial 1 with value: 689.8169979283266.
[I 2025-07-22 04:14:10,910] Trial 2 finished with value: 669.6805520970248 and parameters: {'num_leaves': 54, 'learning_rate': 0.09445645065743215, 'min_child_samples': 63, 'max_depth': 4, 'feature_fr

This code block is dedicated to training the final LightGBM model for Problem Two (forecasting passenger numbers one week in advance) using the optimal hyperparameters identified by Optuna, and then rigorously evaluating its performance on the held-out test set.

In [24]:
final_model_p02 = lgb.train(best_params_p02,
                            lgb.Dataset(X_train_p02, label=y_train_p02),
                            num_boost_round=1000) 

y_pred_final_p02 = final_model_p02.predict(X_test_p02)


rmse_final_p02 = np.sqrt(mean_squared_error(y_test_p02, y_pred_final_p02))
r2_final_p02 = r2_score(y_test_p02, y_pred_final_p02)

print(f"\nFinal Model Performance on X_test_p02:")
print(f"RMSE: {rmse_final_p02:.2f}")
print(f"R²: {r2_final_p02:.4f}")


Final Model Performance on X_test_p02:
RMSE: 632.22
R²: 0.8054


#### ``RMSE: 632.22`` 
On average, this model weekly passenger forecasts are off by approximately 632 passengers.

#### ``R²: 0.8054 ``
This indicates that about ``80.54% ``of the variability in weekly passenger numbers can be explained by this model features and its predictions.

In [25]:
train_allocation_p02= calculate_needed_trains(y_pred_final_p02)
train_allocation_p02.sample(5)

,Predicted_Passengers,Base_Predicted_Trains,Buffered_Predicted_Trains,Final_Predicted_Trains
5054,1650,3,3,3
9269,342,1,1,1
2293,941,2,2,2
2764,121,1,1,1
4798,4422,8,8,8


# Problem Three:
 Assume we need to forecast the number of passengers for different time periods in order to allocate a more accurate number of trains for the next day. (In this case, we are allowed to use information from previous time periods, but using the year and number of trains columns is still not permitted.)


## Splitting data for problem three :

This code defines the preprocess_and_split_data_p03 function, which is designed to prepare your data for Problem Three: forecasting passenger numbers for the next day. This problem emphasizes short-term accuracy and relies heavily on recent historical data, making time-series specific feature engineering and a chronological split essential.
The function takes your DataFrame, performs necessary time-series feature engineering (creating lags and rolling statistics), and then splits the data into training and testing sets based on chronological order.

In [26]:
def preprocess_and_split_data_p03(df, target_column='Ridership', train_size=0.8):
       
    df['Date'] = pd.to_datetime(
    df['Year'].astype(str) + '-' +
    df['Month_Num'].astype(str) + '-' +
    df['Day'].astype(str),
    errors='coerce')

    df.dropna(subset=['Date'], inplace=True)
    df.sort_values(by=['Date'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Identify columns that define a unique time-series for lagging
    # These are the columns that define "which" time series we are lagging.
    grouping_cols = [
      col for col in df.columns if 
      col.startswith('Corridor_') or
      col.startswith('Station_') or
      col.startswith('Period_')]
    
    if 'Workday_y' in df.columns:
       grouping_cols.append('Workday_y')
 
    if 'Workday_n' in df.columns:
       grouping_cols.append('Workday_n')

    if 'Covid19' in df.columns:
       grouping_cols.append('Covid19')

    # Create lagged features for 'Passengers'
    # Lag 1 day (most important for next-day forecast)
    df['Ridership_Lag_1'] = df.groupby(grouping_cols)['Ridership'].shift(1)
    # Lag 2 days
    df['Ridership_Lag_2'] = df.groupby(grouping_cols)['Ridership'].shift(2)
    # Lag 7 days (for weekly seasonality, still important)
    df['Ridership_Lag_7'] = df.groupby(grouping_cols)['Ridership'].shift(7)

    # Rolling Average (e.g., 3-day rolling mean, shifted to avoid leakage)
    # min_periods=1 allows calculation even if fewer than 3 periods are available at the start.
    df['Rolling_Avg_Ridership_3'] = df.groupby(grouping_cols)['Ridership'].transform(
       lambda x: x.rolling(window=3, min_periods=1).mean().shift(1))
    
    # Rolling Standard Deviation (e.g., 3-day rolling std, shifted)
    df['Rolling_Std_Ridership_3'] = df.groupby(grouping_cols)['Ridership'].transform(
         lambda x: x.rolling(window=3, min_periods=1).std().shift(1)).fillna(0)

    df.dropna(subset=['Ridership_Lag_1','Ridership_Lag_2','Ridership_Lag_7','Rolling_Avg_Ridership_3'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Define features (X) and target (y)
    X = df.drop(columns=[target_column, 'Year', 'N_trains', 'Date'])
    y = df[target_column]

    # Calculate split point
    split_point = int(len(df) * train_size)

    # Split the data into training and testing sets
    X_train = X.iloc[:split_point]
    y_train = y.iloc[:split_point]
    
    X_test = X.iloc[split_point:]
    y_test = y.iloc[split_point:]

    print(f"\nTrain set features (X_train) shape: {X_train.shape}")
    print(f"Test set features (X_test) shape: {X_test.shape}")
    print(f"Train set target (y_train) shape: {y_train.shape}")
    print(f"Test set target (y_test) shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test

In [27]:
X_train_p03, X_test_p03, y_train_p03, y_test_p03 = preprocess_and_split_data_p03(df)


Train set features (X_train) shape: (48742, 86)
Test set features (X_test) shape: (12186, 86)
Train set target (y_train) shape: (48742,)
Test set target (y_test) shape: (12186,)


### LightGBM model:

This code defines the objective_p03 function, which serves as the core of the hyperparameter optimization process using the Optuna library for Problem Three: forecasting passenger numbers for the next day. This function is designed to be called repeatedly by Optuna, with each call evaluating a different set of hyperparameters for the LightGBM model.

In [29]:
def objective_p03(trial):

    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 10),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'verbose': -1,
        'feature_pre_filter': False 
    }


    train_data = lgb.Dataset(X_train_p03, label=y_train_p03)
    test_data = lgb.Dataset(X_test_p03, label=y_test_p03, reference=train_data)

 
    model = lgb.train(params,
                      train_data,
                      num_boost_round=1000, 
                      valid_sets=[test_data],
                      callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]) 


    y_pred = model.predict(X_test_p03, num_iteration=model.best_iteration)

    return np.sqrt(mean_squared_error(y_test_p03, y_pred))

This code block initiates and executes the hyperparameter optimization process for Problem Three (forecasting passenger numbers for the next day) using the Optuna library. It orchestrates an intelligent search for the best set of LightGBM hyperparameters that minimize the Root Mean Squared Error (RMSE) on the validation set, aiming for the most accurate next-day forecasts.

In [30]:
study_p03 = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42)) 
study_p03.optimize(objective_p03, n_trials=70, show_progress_bar=True) 

best_params_p03 = study_p03.best_params
best_rmse_optuna_p03 = study_p03.best_value
print("\nBest parameters found by Optuna:", best_params_p03)
print(f"Best RMSE on validation set during tuning (Optuna): {best_rmse_optuna_p03:.2f}")

[I 2025-07-22 04:27:13,088] A new study created in memory with name: no-name-2004b13f-27c2-4790-b331-49293e179ab7


  0%|          | 0/70 [00:00<?, ?it/s]

[I 2025-07-22 04:27:13,769] Trial 0 finished with value: 564.5341626443118 and parameters: {'num_leaves': 50, 'learning_rate': 0.28570714885887566, 'min_child_samples': 75, 'max_depth': 8, 'feature_fraction': 0.5780093202212182, 'bagging_fraction': 0.5779972601681014, 'bagging_freq': 0, 'lambda_l1': 8.661761457749352, 'lambda_l2': 6.011150117432088}. Best is trial 0 with value: 564.5341626443118.
[I 2025-07-22 04:27:18,189] Trial 1 finished with value: 566.8601726273216 and parameters: {'num_leaves': 77, 'learning_rate': 0.01596950334578271, 'min_child_samples': 98, 'max_depth': 11, 'feature_fraction': 0.6061695553391381, 'bagging_fraction': 0.5909124836035503, 'bagging_freq': 2, 'lambda_l1': 3.0424224295953772, 'lambda_l2': 5.247564316322379}. Best is trial 0 with value: 564.5341626443118.
[I 2025-07-22 04:27:18,963] Trial 2 finished with value: 559.2096247150855 and parameters: {'num_leaves': 54, 'learning_rate': 0.09445645065743215, 'min_child_samples': 63, 'max_depth': 4, 'feature_

This code block is dedicated to training the final LightGBM model for Problem Three (forecasting passenger numbers for the next day) using the optimal hyperparameters identified by Optuna, and then rigorously evaluating its performance on the held-out test set. This evaluation provides the most realistic assessment of how well the model will perform on truly unseen future data for next-day forecasts.

In [31]:
final_model_p03 = lgb.train(best_params_p03,
                            lgb.Dataset(X_train_p03, label=y_train_p03),
                            num_boost_round=1000) 

y_pred_final_p03 = final_model_p03.predict(X_test_p03)

rmse_final_p03 = np.sqrt(mean_squared_error(y_test_p03, y_pred_final_p03))
r2_final_p03 = r2_score(y_test_p03, y_pred_final_p03)

print(f"\nFinal Model Performance on X_test_p03:")
print(f"RMSE: {rmse_final_p03:.2f}")
print(f"R²: {r2_final_p03:.4f}")


Final Model Performance on X_test_p03:
RMSE: 565.81
R²: 0.8455


#### ``RMSE:592.89`` 
On average, this model next-day passenger forecasts are off by approximately 593 passengers.

### ``R²:0.8324`` 
This indicates that about 83.24% of the variability in next-day passenger numbers can be explained by this model features and its predictions. 

In [32]:
train_allocation_p03= calculate_needed_trains(y_pred_final_p03)
train_allocation_p03.sample(5)

,Predicted_Passengers,Base_Predicted_Trains,Buffered_Predicted_Trains,Final_Predicted_Trains
6217,748,2,2,2
9026,99,1,1,1
10485,585,1,2,2
1155,183,1,1,1
2979,231,1,1,1
